**Notebook to produce 3-colour AIA movies**

Credit: David Long, DCU

In [1]:
%matplotlib widget
import glob
import datetime as dt
from sunpy.net import Fido, attrs as a
import astropy.units as u
from matplotlib.font_manager import FontProperties
import matplotlib.pyplot as plt
from sunpy.map import Map
from aiapy.calibrate import register, update_pointing, normalize_exposure
import matplotlib.colors as colors
import sunkit_image.enhance as enhance
import numpy as np
from astropy.io import fits as fits
from sunpy.time import parse_time
import cv2
import os
from astropy.time import Time
import scipy.ndimage as ndimage

font0 = FontProperties()
font0.set_size('xx-large')
font0.set_weight('semibold')

In [ ]:
## Function to get closest element in array to defined value
def closest(list, value):
    """
    Get the closest element to value in list

    Parameters
    ----------
    list: Numpy array
        Array of numbers in which to find the value
    value: Number
        Number to find in list
    
    Return: Integer of the number in list closest to value
    """
    ind = np.abs([elem - value for elem in list])
    return ind.argmin(0)

In [2]:
## Define constants
passband = ['171', '193', '211']

start_time = '2023/04/20 04:45:00'
end_time = '2023/04/20 06:45:00'
attrs_time = a.Time(start_time, end_time)

cadence = a.Sample(12*u.second)
aia_instr = a.Instrument('AIA')
aia_provider = a.Provider('JSOC')

file_date = dt.datetime.strftime(dt.datetime.strptime(start_time,'%Y/%m/%d %H:%M:%S'), '%Y/%m/%d')

data_dir = '/Users/dml/Data/SDO/'
output_dir = '/Users/dml/python_output/three_color_movie_images/'+dt.datetime.strftime(dt.datetime.strptime(start_time,'%Y/%m/%d %H:%M:%S'), '%Y%m%d')+'/'
os.makedirs(output_dir, exist_ok='True')

Get the AIA data

In [3]:
## Function to get the list of AIA filenames
def get_aia_filelist(data_dir, passband, file_date):
    files = glob.glob(data_dir+str(passband).rjust(4, "0")+'/'+file_date+'/*.fits', recursive=True)
    files.sort()
    files_dt = []
    for file_i in files:
        hdr = fits.getheader(file_i, 1)
        try:
            files_dt.append(dt.datetime.strptime(hdr.get('DATE-OBS'),'%Y-%m-%dT%H:%M:%S.%fZ'))
        except:
            files_dt.append(dt.datetime.strptime(hdr.get('DATE-OBS'),'%Y-%m-%dT%H:%M:%S.%f'))
    return files, files_dt

In [4]:
# Find the data, downloading it if necessary
f_0171, time_0171 = get_aia_filelist(data_dir, 171, file_date)
if f_0171 == []:
    print('Downloading data for '+str(171).rjust(4, "0")+' passband')
    pband_dir = data_dir+str(171).rjust(4, "0")+'/'+file_date
    os.makedirs(pband_dir, exist_ok='True')
    result = Fido.search(attrs_time, aia_instr, a.Wavelength(int(171)*u.Angstrom, int(171)*u.Angstrom), cadence, aia_provider)
    files = Fido.fetch(result, path = pband_dir, overwrite=False, progress=True)
f_0171, time_0171 = get_aia_filelist(data_dir, 171, file_date)

f_0193, time_0193 = get_aia_filelist(data_dir, 193, file_date)
if f_0193 == []:
    print('Downloading data for '+str(193).rjust(4, "0")+' passband')
    pband_dir = data_dir+str(193).rjust(4, "0")+'/'+file_date
    os.makedirs(pband_dir, exist_ok='True')
    result = Fido.search(attrs_time, aia_instr, a.Wavelength(int(193)*u.Angstrom, int(193)*u.Angstrom), cadence, aia_provider)
    files = Fido.fetch(result, path = pband_dir, overwrite=False, progress=True)
f_0193, time_0193 = get_aia_filelist(data_dir, 193, file_date)

f_0211, time_0211 = get_aia_filelist(data_dir, 211, file_date)
if f_0211 == []:
    print('Downloading data for '+str(211).rjust(4, "0")+' passband')
    pband_dir = data_dir+str(211).rjust(4, "0")+'/'+file_date
    os.makedirs(pband_dir, exist_ok='True')
    result = Fido.search(attrs_time, aia_instr, a.Wavelength(int(211)*u.Angstrom, int(211)*u.Angstrom), cadence, aia_provider)
    files = Fido.fetch(result, path = pband_dir, overwrite=False, progress=True)
f_0211, time_0211 = get_aia_filelist(data_dir, 211, file_date)

n_files = [len(f_0171), len(f_0193), len(f_0211)]
if np.argmin(n_files) == 0:
    file_list = f_0171
    time_list = time_0171
elif np.argmin(n_files) == 1:
    file_list = f_0193
    time_list = time_0193
elif np.argmin(n_files) == 2:
    file_list = f_0211
    time_list = time_0211

Now process the data for each time step and plot the three colour image

In [5]:
def mk_hires_map(map, h_value, smooth):

    img = enhance.mgn(map.data, h=h_value)
    out = ndimage.gaussian_filter(img, smooth, mode='nearest')
    
    alpha = 0.01
    lwr_bnd = np.percentile(out, alpha)
    upr_bnd = np.percentile(out, 100-alpha)

    out[out < lwr_bnd] = lwr_bnd
    out[out > upr_bnd] = upr_bnd
    high_res_map = Map(out, map.meta)

    return high_res_map

In [6]:
## Function to plot map image and save it as a PNG
def plot_truecolour_image(image, map, output_dir):
    ## Get the date and time of the map for the image filename
    t=Time.strftime(map.date,"%Y%m%d_%H%M%S")
    img_t = Time.strftime(map.date,"%Y-%m-%dT%H:%M:%S")

    ## Plot the image
    fig = plt.figure(num=1,figsize=(25,25))
    axes = plt.Axes(fig,[0.,0.,1.,1.])
    axes.text(0.95, 0.05, img_t, verticalalignment='bottom', horizontalalignment='right',
              color='white',transform=axes.transAxes,fontsize='xx-large',fontweight='semibold')
    axes.set_axis_off()
    fig.add_axes(axes)
    im = axes.imshow(image)

    plt.savefig(output_dir+'hires_3colour_image_'+t+'.png',bbox_inches='tight')
    plt.close(fig)

In [7]:
## Function to plot map image and save it as a PNG
def plot_tc_diff_image(image, map, output_dir):
    ## Get the date and time of the map for the image filename
    t=Time.strftime(map.date,"%Y%m%d_%H%M%S")
    img_t = Time.strftime(map.date,"%Y-%m-%dT%H:%M:%S")

    ## Plot the image
    fig = plt.figure(num=1,figsize=(25,25))
    axes = plt.Axes(fig,[0.,0.,1.,1.])
    axes.text(0.95, 0.05, img_t, verticalalignment='bottom', horizontalalignment='right',
              color='white',transform=axes.transAxes,fontsize='xx-large',fontweight='semibold')
    axes.set_axis_off()
    fig.add_axes(axes)
    im = axes.imshow(image)

    plt.savefig(output_dir+'tc_diff_image_'+t+'.png',bbox_inches='tight')
    plt.close(fig)

In [9]:
## Function to save image files as a movie
def convert_frames_to_movie(path_in, path_out, fps):
    frame_array = []
    files = [img for img in os.listdir(path_in) if img.endswith(".png")]
    files.sort()
    for i in range(len(files)):
        frame = cv2.imread(os.path.join(output_dir, files[i]))
        height, width, layers = frame.shape
        size = (width,height)
        frame_array.append(frame)

    out = cv2.VideoWriter(path_out,cv2.VideoWriter_fourcc('m', 'p', '4', 'v'), fps, size)

    for i in range(len(frame_array)):
        out.write(frame_array[i])
    out.release()

    cv2.destroyAllWindows()

To produce high resolution images

In [10]:
for f_num in range(417, len(file_list)):

    ind_171 = closest(parse_time(time_list), parse_time(time_0171[f_num]))
    ind_193 = closest(parse_time(time_list), parse_time(time_0193[f_num]))
    ind_211 = closest(parse_time(time_list), parse_time(time_0211[f_num]))

    f_array = [f_0171[ind_171], f_0193[ind_193], f_0211[ind_211]]
    maps = Map(f_array)
    fov_x = [1024, 2048]
    fov_y = [1024, 2048]

    map_array = []

    for m in range(0, len(maps)):
        aia_map_updated_pointing = update_pointing(maps[m])
        aia_map_registered = register(aia_map_updated_pointing)
        aia_map_norm = normalize_exposure(aia_map_registered)
        submap = aia_map_norm.submap([fov_x[0], fov_y[0]]*u.pixel, top_right=[fov_x[1], fov_y[1]]*u.pixel)
        if m == 0:
            map_array.append(mk_hires_map(submap, 0.915, 2))
        else: map_array.append(mk_hires_map(submap, 0.94, 2))

    true_colour_img = (np.dstack([map_array[0].data, map_array[1].data, map_array[2].data])* 255).astype(np.uint8)

    plot_truecolour_image(true_colour_img, map_array[0], output_dir)

## Make a movie of the processed images
video_name = dt.datetime.strftime(dt.datetime.strptime(start_time,'%Y/%m/%d %H:%M:%S'), '%Y%m%d')+'_three_colour.mp4'
convert_frames_to_movie(output_dir, output_dir+video_name, 15.0)


To produce running difference images

In [ ]:
offset = 60*u.second
ind_start = closest(parse_time(time_list), (parse_time(time_list[0])+offset))

diff_rng = [-20, 20]

f_num = ind_start
#for f_num in range(ind_start, len(file_list)):

ind_171 = closest(parse_time(time_list), (parse_time(time_0171[f_num])-offset))
ind_193 = closest(parse_time(time_list), (parse_time(time_0193[f_num])-offset))
ind_211 = closest(parse_time(time_list), (parse_time(time_0211[f_num])-offset))

f_arr_0 = [f_0171[ind_171], f_0193[ind_193], f_0211[ind_211]]
maps_0 = Map(f_arr_0)

ind_171 = closest(parse_time(time_list), parse_time(time_0171[f_num]))
ind_193 = closest(parse_time(time_list), parse_time(time_0193[f_num]))
ind_211 = closest(parse_time(time_list), parse_time(time_0211[f_num]))

f_arr_1 = [f_0171[ind_171], f_0193[ind_193], f_0211[ind_211]]
maps_1 = Map(f_arr_1)

fov_x = [1024, 2048]
fov_y = [1024, 2048]

map_array_0 = []
map_array_1 = []

for m in range(0, len(maps_1)):
    aia_map_updated_pointing = update_pointing(maps_0[m])
    aia_map_registered = register(aia_map_updated_pointing)
    aia_map_norm = normalize_exposure(aia_map_registered)
    submap_0 = aia_map_norm.submap([fov_x[0], fov_y[0]]*u.pixel, top_right=[fov_x[1], fov_y[1]]*u.pixel)
    map_array_0.append(submap_0)

    aia_map_updated_pointing = update_pointing(maps_1[m])
    aia_map_registered = register(aia_map_updated_pointing)
    aia_map_norm = normalize_exposure(aia_map_registered)
    submap_1 = aia_map_norm.submap([fov_x[0], fov_y[0]]*u.pixel, top_right=[fov_x[1], fov_y[1]]*u.pixel)
    map_array_1.append(submap_1)

tc_img_0 = np.dstack([map_array_0[0].data, map_array_0[1].data, map_array_0[2].data])
tc_img_1 = np.dstack([map_array_1[0].data, map_array_1[1].data, map_array_1[2].data])

d_img = tc_img_1-tc_img_0

#out[out > upr_bnd] = upr_bnd
#out[out < lwr_bnd] = lwr_bnd
#d_img = (d_img* 255).astype(np.uint8)
#high_res_map = Map(out, map.meta)



#plot_tc_diff_image(d_img, map_array[0], output_dir)
#
### Make a movie of the processed images
#video_name = dt.datetime.strftime(dt.datetime.strptime(start_time,'%Y/%m/%d %H:%M:%S'), '%Y%m%d')+'_three_colour_diff.mp4'
#convert_frames_to_movie(output_dir, output_dir+video_name, 15.0)